In [1]:
import pandas as pd
import os as os
import numpy as np
import talib as talib
import json as json
from datetime import datetime, timedelta, date, time
from os import walk
pd.options.mode.chained_assignment = None

In [2]:
"""
1.0 传叁
file1 = request.form.get('file1') # 文件1
file2 = request.form.get('file2') # 文件2
direction = request.form.get('direction') # 方向
vol = float(request.form.get('vol'))  # 买卖波幅
vol2 = float(request.form.get('vol2'))  # 止蚀波幅
cutloss = float(request.form.get('cutLoss'))  # 止蚀
is_adjust = False if request.form.get('adjection')=='false' else True # 调整
minutes = int(request.form.get('minutes')) # 预测x分钟后
"""

# 1.0 传叁
file1 = '20201210-181232'
file2 = 't5-0608-0922-e12-u512-w50-lr0.0005'
direction = 'long' # 方向
vol = 0.35 # 买卖波幅
vol2 = 0.0 # 止蚀波幅
cutloss = 1.50 # 止蚀
is_adjust = False # 调整
minutes = 5 # 预测x分钟后

# 1.1 数据集
path1 = os.path.abspath(os.path.join('data', 'nq', 'prediction', file1, file2+'.csv'))
df1 = pd.read_csv(path1)
df1.udate = pd.to_datetime(df1.udate)
df1.index = pd.to_datetime(df1.udate)
df1

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5
udate,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,2020-06-08 17:02:00,9860.25,9860.50,9859.25,9859.50,5.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,2020-06-08 17:03:00,9859.50,9860.00,9857.50,9857.50,12.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,2020-06-08 17:04:00,9857.50,9859.25,9857.50,9859.25,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11150.50,11149.50,11149.75,23.0,11154.694818,11154.617827,11153.598514,11154.442383,11154.586296
2020-09-22 15:50:00,2020-09-22 15:50:00,11149.75,11151.75,11148.50,11151.75,77.0,11155.846154,11155.777841,11154.731506,11155.608944,11155.712494
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.75,11152.50,11150.75,11151.25,43.0,11157.970479,11157.926811,11156.847524,11157.766746,11157.826288


In [3]:
# 2.0 数据集2
df2 = df1.copy(deep=True)

highs = np.array(df2['High'], dtype='float')
lows = np.array(df2['Low'], dtype='float')
opens = np.array(df2['Open'], dtype='float')
closes = np.array(df2['Close'], dtype='float')
vols = np.array(df2['Volume'], dtype='float')
df2['atr'] = talib.ATR(highs, lows, closes, timeperiod=14*2)

df2['last1'] = df2['Close'].shift(periods=-1) # 上笔数据 最后价
df2['adjust'] = df2['Close'] - df2['last1'] # 前1分钟 结算价 波幅

key2 = 't'+str(minutes)
if not is_adjust:
    df2['plast'] = df2[key2] # 预测 5分钟后的 波幅 
    df2['plast1'] = df2['t1'] # 预测 1分钟后的 波幅
else:
    df2['plast'] = df2[key2] + df2['adjust'] # 已调整
    df2['plast1'] = df2['t1'] + df2['adjust'] # 已调整

df2['pchange'] = (df2['plast'] - df2['Close']) / df2['Close']*100 # 预测 5分钟后的 波幅率
df2['pchange1'] = (df2['plast1'] - df2['Close']) / df2['Close']*100 # 预测 1分钟后的 波幅率

df2['real(t+1)'] = df2['Close'].shift(periods=-1)
df2['real(t+5)'] = df2['Close'].shift(periods=-5)

df2

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5,atr,last1,adjust,plast,plast1,pchange,pchange1,real(t+1),real(t+5)
udate,,,,,,,,,,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9865.50,0.00,NaN,NaN,NaN,NaN,9865.50,9859.0
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.50,6.00,NaN,NaN,NaN,NaN,9859.50,9860.0
2020-06-08 17:02:00,2020-06-08 17:02:00,9860.25,9860.50,9859.25,9859.50,5.0,NaN,NaN,NaN,NaN,NaN,NaN,9857.50,2.00,NaN,NaN,NaN,NaN,9857.50,9861.5
2020-06-08 17:03:00,2020-06-08 17:03:00,9859.50,9860.00,9857.50,9857.50,12.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.25,-1.75,NaN,NaN,NaN,NaN,9859.25,9861.0
2020-06-08 17:04:00,2020-06-08 17:04:00,9857.50,9859.25,9857.50,9859.25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.00,0.25,NaN,NaN,NaN,NaN,9859.00,9862.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11150.50,11149.50,11149.75,23.0,11154.694818,11154.617827,11153.598514,11154.442383,11154.586296,2.762027,11151.75,-2.00,11154.586296,11154.694818,0.043376,0.044349,11151.75,NaN
2020-09-22 15:50:00,2020-09-22 15:50:00,11149.75,11151.75,11148.50,11151.75,77.0,11155.846154,11155.777841,11154.731506,11155.608944,11155.712494,2.734812,11151.25,0.50,11155.712494,11155.846154,0.035532,0.036731,11151.25,NaN
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.75,11152.50,11150.75,11151.25,43.0,11157.970479,11157.926811,11156.847524,11157.766746,11157.826288,2.663926,11151.00,0.25,11157.826288,11157.970479,0.058974,0.060267,11151.00,NaN


In [4]:
# 3.0 回测
df4 = pd.DataFrame(columns = ['udate', 'action', 'cash', 'profit', 'message'])
holding = False # 是否持有仓位
top_profit_price = 0 # 买入点数
fee = 2.1 # 交易费
cash = 0 # 资金流

def is_trigger(v2, vol):
    if direction == 'long':
        return (v2['pchange'] >= vol)
    elif direction == 'short':
        return (v2['pchange'] <= -vol)

def is_stop_profit(v2, top_profit_price, direction):
    if direction == 'long':
        return (abs(v1['Close']-top_profit_price) >= abs(cutloss*v1['atr']))
    elif direction == 'short':
        return (abs(top_profit_price-v1['Close']) >= abs(cutloss*v1['atr']))
    
def is_cut_loss(v2, vol2):
    if direction == 'long':
        return (v2['pchange1'] <= vol2)
    elif direction == 'short':
        return (v2['pchange1'] >= vol2)

df3 = df2.copy(deep=True)
for k1, v1 in df3.iterrows():
    if v1['udate'].hour < 15 or v1['udate'].hour >= 19:
        # 3.2 买入
        if is_trigger(v1, vol) and not holding:
            holding = True
            cost1 = v1['Close']
            top_profit_price = v1['Close']
            action = 'buy' if direction == 'long' else 'sell'
            start_hold_time = k1
            df4 = df4.append({'udate': k1, 'action': action, 'cash': np.nan, 'profit': np.nan, 'hold time': np.nan, 'message': np.nan}, ignore_index=True)
            continue
        # 3.3 卖出
        cut_loss2 = is_cut_loss(v1, vol2) # 止蚀
        stop_profit2 = is_stop_profit(v1, top_profit_price, direction) # 止盈
        cur_time, close_time = time(v1.udate.hour, v1.udate.minute, 0), time(14, 59, 0)
        is_over_night = (cur_time == close_time) # 过夜
        if (cut_loss2 or stop_profit2 or is_over_night) and holding:
            holding = False
            # 3.4 持有时长
            mins_diff = (k1 - start_hold_time).total_seconds() / 60
            # 3.5 盈亏
            if direction == 'long':
                profit2 = (v1['Close']-cost1)*20 - (fee*2)
            elif direction == 'short':
                profit2 = (cost1-v1['Close'])*20 - (fee*2)
            if profit2 > 0:
                profit3 = '<span class="text-danger">+'+str(profit2)+'</span>'
            elif profit2 <= 0:
                profit3 = '<span class="text-success">'+str(profit2)+'</span>'
            message = '('+str(cost1)+' - '+str(v1['Close'])+')*20 - ('+str(fee)+'*2) = '+profit3
            # 3.6 长仓
            if direction == 'long' and stop_profit2:
                action = 'sell'
            elif direction == 'long' and cut_loss2:
                action = 'cut'
            elif direction == 'long' and is_over_night:
                action = 'cut overnight'
            # 3.7 短仓
            elif direction == 'short' and stop_profit2:
                action = 'buy'
            elif direction == 'short' and cut_loss2:
                action = 'cut'
            elif direction == 'short' and is_over_night:
                action = 'cut overnight'
            # 3.8 资金流
            cash = cash + profit2
            
            df4 = df4.append({'udate': k1, 'action': action, 'cash': cash, 'profit': profit2, 'hold time': mins_diff, 'message': message}, ignore_index=True)
            continue

df4.index = df4['udate']
df4.drop(['udate'], axis=1, inplace=True)
df4

,action,cash,profit,message,hold time
udate,,,,,
2020-06-12 09:51:00,buy,NaN,NaN,NaN,NaN
2020-06-12 09:53:00,sell,-374.2,-374.2,(9681.75 - 9663.25)*20 - (2.1*2) = <span class...,2.0
2020-06-12 10:03:00,buy,NaN,NaN,NaN,NaN
2020-06-12 10:04:00,sell,-798.4,-424.2,"(9606.0 - 9585.0)*20 - (2.1*2) = <span class=""...",1.0
2020-06-12 10:19:00,buy,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-09-11 10:30:00,sell,2556.2,-444.2,(11166.0 - 11144.0)*20 - (2.1*2) = <span class...,9.0
2020-09-11 14:37:00,buy,NaN,NaN,NaN,NaN
2020-09-11 14:48:00,cut,2767.0,210.8,(11062.75 - 11073.5)*20 - (2.1*2) = <span clas...,11.0


In [5]:
# 4.0 清理数据集
df5 = pd.concat([df2, df4], axis=1)
drop_list_5 = ['High', 'Low', 'Open', 'Volume', 't2', 't3', 't4', 'last1']
df5.drop(drop_list_5, axis=1, inplace=True)
df5

,udate,Close,t1,t5,atr,adjust,plast,plast1,pchange,pchange1,real(t+1),real(t+5),action,cash,profit,message,hold time
udate,,,,,,,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,9865.50,9859.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,9859.50,9860.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,2020-06-08 17:02:00,9859.50,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,9857.50,9861.5,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,2020-06-08 17:03:00,9857.50,NaN,NaN,NaN,-1.75,NaN,NaN,NaN,NaN,9859.25,9861.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,2020-06-08 17:04:00,9859.25,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,9859.00,9862.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11154.694818,11154.586296,2.762027,-2.00,11154.586296,11154.694818,0.043376,0.044349,11151.75,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-22 15:50:00,2020-09-22 15:50:00,11151.75,11155.846154,11155.712494,2.734812,0.50,11155.712494,11155.846154,0.035532,0.036731,11151.25,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.25,11157.970479,11157.826288,2.663926,0.25,11157.826288,11157.970479,0.058974,0.060267,11151.00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 5.0 渲染1
df6 = df5.loc[(df5['action']=='buy') | (df5['action']=='sell') | (df5['action']=='cut') | (df5['action']=='cut overnight')]
df6.columns = ['udate', 'last', 'predict1(T+1)', 'predict1(T+5)', 'atr', 'shift', 'predict2(T+1)', 'predict2(T+5)', 'p-percent1', 'p-percent5', 'real(t+1)', 'real(t+5)', 
               'action', 'cash', 'profit', 'message', 'hold time']
df6['cash'] = df6['cash'].round(4)
df6['profit'] = df6['profit'].round(2)
# 5.1 渲染2
if direction == 'long':
    no_trade = df6.loc[(df6['action']=='buy')].shape[0]
elif direction == 'short':
    no_trade = df6.loc[(df6['action']=='sell')].shape[0]
total_profit = df6.iloc[-1]['cash'].round(2)
avg_profit = df6['profit'].mean().round(2)
max_win = df6['profit'].max().round(2)
max_loss = df6['profit'].min().round(2)
no_win = int(np.sum(df6['profit'] > 0))
no_loss = int(np.sum(df6['profit'] <= 0))
no_cut_loss = int(np.sum(df6['action'] == 'cut')) + int(np.sum(df6['action'] == 'cut overnight'))
no_total_day = len(df5.groupby(df5['udate'].dt.date).size())
no_trade_day = len(df5.groupby(df6['udate'].dt.date).size())
avg_no_trade = round(no_trade/no_total_day, 2)
hold_time_max = df6['hold time'].max().round(0)
hold_time_min = df6['hold time'].min().round(0)
hold_time_avg = (df6['hold time'].sum()/no_trade).round(0)

# 5.2 加颜色
for k5, v5 in df6.iterrows():
    # 5.3 profit
    if df6.loc[k5, 'profit'] > 0:
        df6.loc[k5, 'profit'] = '<font class="text-danger">+'+str(df6.loc[k5, 'profit'])+'</font>'
    elif df6.loc[k5, 'profit'] <= 0:
        df6.loc[k5, 'profit'] = '<font class="text-success">'+str(df6.loc[k5, 'profit'])+'</font>'
    # 5.4 cash
    if df6.loc[k5, 'cash'] > 0:
        df6.loc[k5, 'cash'] = '<font class="text-danger">+'+str(df6.loc[k5, 'cash'])+'</font>'
    elif df6.loc[k5, 'cash'] <= 0:
        df6.loc[k5, 'cash'] = '<font class="text-success">'+str(df6.loc[k5, 'cash'])+'</font>'
html = df6.to_html(classes='table table-sm table-striped', index=False, escape=False, border=0, justify='left').replace('border="1"', 'border="0"').replace('NaN', '')
data4 = {'total_profit': total_profit, 'avg_profit': avg_profit, 'max_win': max_win, 'max_loss': max_loss,
         'no_trade': no_trade, 'no_win': no_win, 'no_loss': no_loss, 'no_cut_loss': no_cut_loss, 'avg_no_trade': avg_no_trade, 'no_total_day': no_total_day, 
         'hold_time_max': hold_time_max, 'hold_time_min': hold_time_min, 'hold_time_avg': hold_time_avg,
         'html': html}
data5 = json.dumps(data4)

del data4['html']
print(data4)
df6

{'total_profit': 2277.8, 'avg_profit': 7.83, 'max_win': 1050.8, 'max_loss': -1459.2, 'no_trade': 291, 'no_win': 143, 'no_loss': 148, 'no_cut_loss': 4, 'avg_no_trade': 5.29, 'no_total_day': 55, 'hold_time_max': 241.0, 'hold_time_min': 1.0, 'hold_time_avg': 9.0}


,udate,last,predict1(T+1),predict1(T+5),atr,shift,predict2(T+1),predict2(T+5),p-percent1,p-percent5,real(t+1),real(t+5),action,cash,profit,message,hold time
udate,,,,,,,,,,,,,,,,,
2020-06-12 09:51:00,2020-06-12 09:51:00,9681.75,9716.316821,9716.102616,5.054177,5.25,9716.102616,9716.316821,0.354818,0.357031,9676.50,9660.00,buy,NaN,NaN,NaN,NaN
2020-06-12 09:53:00,2020-06-12 09:53:00,9663.25,9694.931131,9694.583078,5.036983,-3.25,9694.583078,9694.931131,0.324250,0.327852,9666.50,9647.50,sell,"<font class=""text-success"">-374.2</font>","<font class=""text-success"">-374.2</font>",(9681.75 - 9663.25)*20 - (2.1*2) = <span class...,2.0
2020-06-12 10:03:00,2020-06-12 10:03:00,9606.00,9640.437541,9639.979302,4.896574,21.00,9639.979302,9640.437541,0.353730,0.358500,9585.00,9647.50,buy,NaN,NaN,NaN,NaN
2020-06-12 10:04:00,2020-06-12 10:04:00,9585.00,9628.934819,9628.432768,4.721696,-32.00,9628.432768,9628.934819,0.453133,0.458371,9617.00,9653.00,sell,"<font class=""text-success"">-798.4</font>","<font class=""text-success"">-424.2</font>","(9606.0 - 9585.0)*20 - (2.1*2) = <span class=""...",1.0
2020-06-12 10:19:00,2020-06-12 10:19:00,9572.50,9615.035688,9614.787328,6.936418,-25.50,9614.787328,9615.035688,0.441758,0.444353,9598.00,9567.00,buy,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-11 10:30:00,2020-09-11 10:30:00,11144.00,11159.977636,11159.808090,8.585868,5.00,11159.808090,11159.977636,0.141853,0.143374,11139.00,11119.75,sell,"<font class=""text-danger"">+2556.2</font>","<font class=""text-success"">-444.2</font>",(11166.0 - 11144.0)*20 - (2.1*2) = <span class...,9.0
2020-09-11 14:37:00,2020-09-11 14:37:00,11062.75,11102.212491,11102.641729,8.793675,-2.75,11102.641729,11102.212491,0.360595,0.356715,11065.50,11061.50,buy,NaN,NaN,NaN,NaN
2020-09-11 14:48:00,2020-09-11 14:48:00,11073.50,11071.777571,11071.860955,8.931270,12.25,11071.860955,11071.777571,-0.014802,-0.015555,11061.25,11056.50,cut,"<font class=""text-danger"">+2767.0</font>","<font class=""text-danger"">+210.8</font>",(11062.75 - 11073.5)*20 - (2.1*2) = <span clas...,11.0
